In [59]:
import nltk

# Download the necessary resources
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize as thai_word_tokenize
from pythainlp.tokenize import syllable_tokenize
from collections import Counter
# from rank_bm25 import BM25Okapi
import pandas as pd
import re
import string
import requests
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from string import digits
from pythainlp.util import abbreviation_to_full_text
en_stop = set(stopwords.words('english'))
th_stop = set(thai_stopwords())

[nltk_data] Downloading package omw-1.4 to /Users/kong/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
df = pd.read_csv('qa_data.csv',header=1)
df.columns = ['Index', 'Topic', 'Question', 'Answer']
# df = df.drop(index=0).reset_index(drop=True)
df = df.iloc[:-1,1:]
df.head()

,Topic,Question,Answer
0,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,บุตรข้าราชการ อายุ 18 ปี แต่งงาน จดทะเบียนสมรส...,ไม่สามารถใช้สิทธิเบิกข้าราชการได้ เนื่องจากบรร...
1,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,บุตรข้าราชการบรรลุนิติภาวะ ไม่สามารถใช้สิทธิเบ...,1.บุตรอายุครบ 20 ปีบริบูรณ์\n2.บุตรแต่งงาน จดท...
2,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,ข้าราชการเกษียณอายุได้รับบำเหน็จ แต่ไม่สามารถห...,หนังสือรับรองที่ออกโดยกำนันหรือผู้ใหญ่บ้านหรือ...
3,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,ข้าราชการกรมบัญชีกลาง หมดสิทธิแล้ว ต้องการปรับ...,1.ติดต่อนายทะเบียนต้นสังกัดเพื่อส่งข้อมูลการหม...
4,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,พนักงานองค์กรปกครองส่วนท้องถิ่น (อปท.) หมดสิทธ...,1330 ไม่สามารถปรับปรุงฐานสิทธิขององค์กรปกครองส...


In [142]:
import requests
 
url = "https://api.aiforthai.in.th/tpos"
 
headers = {
    'Apikey': "uBqPAel8u1k3wE7qeMVHnz1Wpngj6VDR",
    }

def find_pattern_index(lst, pattern):
    for i, value in enumerate(lst):
        if re.match(pattern, value):
            return i
    return ''

def clean(text):
  text = text.translate(str.maketrans('''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~''', ' ' * len('''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~''')))
  # text = re.sub(r'[^\w\s]', '', text)
  text =text.lower()
  text = text.replace('\n', '')
  text = text.replace('\t', '')
  text = text.replace('\r', '')
  text = text.translate(str.maketrans('', '', digits))

  return text


def split_word(text):
    # Tokenize the text
    # tokens = thai_word_tokenize(text, engine='icu', keep_whitespace= False)
    params = {'text':text}
    tokens = requests.get(url, headers=headers, params=params)
 
    return tokens.json()

    # Iteratively process tokens
    # return tokens


def charge_form(text):
  tokens = split_word(text)
  words = tokens['words']
  tags = tokens['tags']
  # print(tags)
  # print(words)
  pattern = r'CNJ|P'
  idx = find_pattern_index(tags, pattern)

  if len(words)> 2:
    if idx != '':
      return f"{text}|{''.join(words[1:idx])}"
    else:
      return f"{text}|{''.join(words[1:])}"
  else:
    return text


In [144]:
import pandas as pd

# Sample data
data = df['clean_Q'].to_list()
text = input('query key words : ')
patterns = clean(text).strip().split()
# patterns = [charge_form(i) for i in patterns]
# print(patterns)

# Create a pandas Series
def search(patterns, data):
    series = pd.Series(data)

    # Create a DataFrame to store the counts
    counts_df = pd.DataFrame(index=series.index)
    nvocab = len(patterns)
    # Count occurrences of each pattern in each string
    for pattern in patterns:
        
        counts_df[pattern] = series.str.count(pattern)

    logit = counts_df > 0
    counts_df = counts_df/logit.sum(axis=0)
    counts_df['tier'] = logit.sum(axis=1)
    counts_df['total_words'] = counts_df.iloc[:,:-1].sum(axis=1)
    counts_df['logit'] = counts_df['tier'] == nvocab
    return counts_df.sort_values(by = ['tier', 'total_words'], ascending = [False, False])

table = search(patterns, data)
s_index = table[table['logit']==True].index.to_list()
if s_index:
    for i in s_index:
        print('Question : ', clean(df['Question'][i]))
        print('Answer :\n', df['Answer'][i].strip())
        print('')
else:
    print('Not found')

['ไต', 'บัตรทอง']
Not found
